# ***<u>librariess***

In [31]:
import numpy as np
from itertools import product
from collections import defaultdict
import copy

# ***<u>Generate <u>Sudoku***

In [32]:
def generate_sudoku():

    """
        Function to generate a Sudoku grid
        Fill the diagonal boxes of the Sudoku grid
        Solve the Sudoku grid
        Remove elements from the grid to create a puzzle
    """
    grid = [[0 for _ in range(9)] for _ in range(9)]
    fill_diagonal(grid)
    # display_grid(grid)
    steps_count = 0
    _ , steps_count = solve_sudoku(grid, steps_count)
    num_elements = int(input("How many cells of the origin grid you want to remove?: "))
    incomplete_grid = remove_elements(grid, num_elements)
    return incomplete_grid



def fill_diagonal(grid):
    
    """
        Function to fill the diagonal boxes of the Sudoku grid
    """
    ###
    fill_box(grid, 0, 0)
    fill_box(grid, 3, 3)
    fill_box(grid, 6, 6)


def fill_box(grid, row, col):

    """
        Function to fill a box with random values
    """
    domain = list(range(1, 10))
    np.random.shuffle(domain)
    for i in range(3):
        for j in range(3):
            num = 0
            for n in range(1, 10):
                #check if the num is safe for our box
                if is_safe(grid, row + i, col + j, domain[n - 1]):
                    num = domain[n - 1]
                    break
            grid[row + i][col + j] = num


def is_safe(grid, row, col, num):

    """
        Function to check if it is safe to place a number in a particular position
    """

    return (
        not used_in_row(grid, row, num)
        and not used_in_column(grid, col, num)
        and not used_in_box(grid, row - row % 3, col - col % 3, num)
    )


# 
def used_in_row(grid, row, num):

    """
        Function to check if a number is used in a row
    """
    found = False
    if num in grid[row]:
        found = True
    return found


def used_in_column(grid, col, num):

    """
        Function to check if a number is used in a column
    """
    found = False
    for i in range (0, 9):
        if grid[i][col] == num:
            found = True
            break
    return found


def used_in_box(grid, box_start_row, box_start_col, num):

    """
        Function to check if a number is used in a 3x3 box
    """
    found = False
    for i in range(3):
        for j in range(3):
            if grid[box_start_row + i][box_start_col + j] == num:
                found = True
                break
    return found

def find_unassigned_location(grid):

    """
        Function to find an unassigned location in the grid
    """  
    empty_cell = (-1, -1)
    for i in range(9):
        for j in range(9):
            if grid[i][j] == 0:
                empty_cell = (i, j)
                return empty_cell
    return empty_cell
    

def remove_elements(grid, num_elements):
    
    """
        Function to remove elements from the grid
    """
    for i in range(num_elements):
        removing = True 
        while removing:
            r = np.random.randint(0, 9)
            c = np.random.randint(0, 9)
            if grid[r][c] != 0:
                grid[r][c] = 0
                removing = False
    return grid


# ***<u>BackTracking***

In [33]:
def solve_sudoku(grid, steps_count):
    """
    Function to solve the Sudoku grid using backtracking
    """
    cell = find_unassigned_location(grid)  # if all cells were filled correctly, return from the call
    if cell == (-1, -1):
        return True, steps_count

    for n in range(1, 10):  # check all numbers 1, 2, ..., 9
        if is_safe(grid, cell[0], cell[1], n) == False:  # if this number is not safe to fill the empty cell, check next number
            continue
        grid[cell[0]][cell[1]] = n  # assign that number to the empty cell
        steps_count += 1
        solved, steps_count = solve_sudoku(grid, steps_count)  # recursively solve the new state
        if solved:
            return True, steps_count
        grid[cell[0]][cell[1]] = 0  # backtrack if we reached a dead end

    return False, steps_count
    
    

def display_grid(grid):

    """
        Function to display the Sudoku grid
    """

    for i in range(9):
        for j in range(9):
            print(grid[i][j], end=" ")
        print()

# ***<u>CSP***

In [34]:
def solve_sudoku_csp(grid):
    """
        Function to solve the Sudoku grid using Constraint Satisfaction Problem (CSP)
    """

    def create_domains(grid):
        """
            Function to create domains for each cell in the grid
        """
        assignment = {}
        for r in range(9):
            for c in range(9):
                if grid[r][c] == 0:
                    domain = []
                    for i in range(1, 10):
                        if is_safe(grid, r, c, i):
                            domain.append(i)
                    assignment[(r, c)] = domain
                else:
                    assignment[(r, c)] = grid[r][c]
        return assignment
            

    def is_valid_assignment(i, j, val, assignment):
        for col in range(9):
            if (assignment[(i, col)] == {val}):
                return False

        for row in range(9):
            if (assignment[(row, j)] == {val}):
                return False
            
        box_row, box_col = i - i % 3, j - j % 3
        for row in range(box_row, box_row + 3):
            for col in range(box_col, box_col + 3):
                if (assignment[(row, col)] == {val}):
                    return False
        return True



    def find_unassigned_location(assignment):
        """
            Function to find an unassigned location in the grid
        """
        empty_cell = (-1, -1)
        for i in range(9):
            for j in range(9):
                if type(assignment[(i, j)]) == list:
                    empty_cell = (i, j)
                    return empty_cell
        return empty_cell
    
    def forward_checking(row, col, assignment, val):
        edited_cells = [] 
        for i in range(9):
            if type(assignment[(row, i)]) == list and val in assignment[(row, i)]:
                assignment[(row, i)].remove(val)
                edited_cells.append((row, i))

        for j in range(9):
            if type(assignment[(j, col)]) == list and val in assignment[(j, col)]:
                assignment[(j, col)].remove(val)
                edited_cells.append((j, col))

        box_i = 3 * (row // 3)
        box_j = 3 * (col // 3) 
        for i in range(box_i, box_i + 3):
            for j in range(box_j, box_j + 3):
                if type(assignment[(i, j)]) == list and val in assignment[(i, j)]:
                    assignment[(i, j)].remove(val)
                    edited_cells.append((i, j))
        return edited_cells
        

    def undo_forward_checking(assignment, val, edited_cells):
        for r, c in edited_cells:
            assignment[(r, c)].append(val)

    steps_count = 0
    def solve_csp(assignment):
        nonlocal steps_count
        (i, j) = find_unassigned_location(assignment)
        if (i, j) == (-1,-1):
            print(f"steps = {steps_count}\n")
            return True

        copy_domain = assignment[(i, j)].copy()
        for val in copy_domain:
            if not is_valid_assignment(i, j, val, assignment):
                continue
            assignment[(i, j)] = val
            edited_cells = forward_checking(i, j, assignment, val)
            steps_count += 1
            if solve_csp(assignment):
                return True
            undo_forward_checking(assignment, val, edited_cells)
        assignment[(i, j)] = copy_domain
        return False

    assignment = create_domains(grid)
    if solve_csp(assignment):
        solved_grid = [[assignment[(i, j)] for j in range(9)] for i in range(9)]
        return solved_grid
    else:
        return None

# ***<u>Show <u>Result***

In [35]:
# Function to initialize the Sudoku grid
def initializing_grid():
    print("\nInitial Sudoku\n")
    sudoku_grid = generate_sudoku()
    for i in sudoku_grid:
        for j in i:
            print(j, end=' ')
        print()
    print(end='\n\n\n\n\n')
    return sudoku_grid


# Function to solve the Sudoku grid using backtracking
def backtracking_answer(sudoku_grid):
    print("Back Tracking Answer:\n\n")
    steps_count = 0
    status, steps_count = solve_sudoku(sudoku_grid, steps_count)
    if status:
        print("Sudoku solved successfully:")
        print()
        print(f"steps: {steps_count}")
        print()
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')


# Function to solve the Sudoku grid using CSP
def csp_answer(sudoku_grid):
    print("CSP Answer:\n")
    solved_grid = solve_sudoku_csp(sudoku_grid)
    if solved_grid is not None:
        print("Sudoku solved successfully:")
        for row in solved_grid:
            for r1 in row:
                print(r1, end=' ')
            print()
    else:
        print("No solution exists for the given Sudoku.")

    # print(f"moves: {moves}")

In [36]:
# Generate and display the initial Sudoku grid
generate_sudoko = initializing_grid()


# # Solve the Sudoku grid using backtracking
backtracking_answer(copy.deepcopy(generate_sudoko))


# # Solve the Sudoku grid using CSP
csp_answer(generate_sudoko)


Initial Sudoku

0 0 4 2 3 0 0 0 0 
0 0 0 0 0 0 0 0 3 
0 7 3 5 8 0 0 2 6 
0 1 0 0 0 5 0 0 0 
3 0 0 0 0 8 2 6 5 
0 5 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 
0 0 2 0 0 0 0 0 0 
0 0 8 0 0 0 3 0 0 





Back Tracking Answer:


Sudoku solved successfully:

steps: 208659

1 8 4 2 3 6 5 7 9 
6 2 5 1 7 9 4 8 3 
9 7 3 5 8 4 1 2 6 
2 1 6 3 9 5 7 4 8 
3 4 9 7 1 8 2 6 5 
8 5 7 4 6 2 9 3 1 
4 3 1 6 5 7 8 9 2 
5 9 2 8 4 3 6 1 7 
7 6 8 9 2 1 3 5 4 





CSP Answer:



KeyboardInterrupt: 